In [1]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cross_validation, metrics, cluster
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#зургийн шинж чанарын матриц ба лейбелүүдийг агуулах өгөгдлийн бүтэц
class Data:
    def __init__(this, records, targets):
        this.records = records
        this.targets = targets

In [3]:
#Хавтсаар уншхад бэлтгэх
filenames = [img for img in glob.glob("pie 10/*.bmp")]
filenames.sort()
#Боловсруулалт
MAX_ILLUMINATION = 255 
X_FEATURES = np.zeros(shape=(150, 2500)) #бүх хавтгайлсан зурагнуудын хүснэгт үүсгэх
for index in range(0, len(filenames)):
    image = cv2.imread(filenames[index], cv2.IMREAD_GRAYSCALE)
    image = cv2.equalizeHist(image) #уншсан зургийнхаа хистограмыг тэнцүүлэх
    image = cv2.resize(image, (50, 50), cv2.INTER_NEAREST) #зургийн хэмжээг багасгах
    X_FEATURES[index] = image.flatten()/MAX_ILLUMINATION #зургийн пикселүүдийн матрицыг хавтайлаад 0-1 утга руу багасгах

In [4]:
#Хэмжээс багасгах процесс
K = 50 #багасгахаар зорьж буй хэмжээс
SIGMA = np.matmul(X_FEATURES.transpose(), X_FEATURES) / 150 #шинж чанарын хэмжээсүүдийн коварианц матриц
P, D, Q = np.linalg.svd(SIGMA, full_matrices=True) #коварианц матрицыг декомпосаци хийх
P_reduced = P[:, 0:K] #эхний к ширхэг баганыг авах
del(P, D, Q)

In [5]:
#Зурагнуудад лейбелийг нь оноох
Z_features = np.matmul(X_FEATURES, P_reduced) #к хэмжээст матрицаа эх зургуудтай үржиж зорилтот матрицаа гаргаж авах
targets = np.array([int(lbl/10)+1 for lbl in range(0, 150)]) #150 зураг тус бүрт 1-15 хүртэл 

data = Data(Z_features, targets) #к хэмжээстэд буулгасан зураг ба тэдгээрийн жинхэнэ хариу

In [8]:
f1_gauss = []
f1_rf = []
f1_knn = []

#К-нугалааст хөндлөн үнэлгээ
kf = cross_validation.KFold(len(data.records), n_folds=5, shuffle=True) #150 өгөгдлийг 30-с бүрдэх 5 хэсэгт хуваах

#нугалаа бүрийн хувьд сургалтын ба туршилтын өгөгдлийг солбих
#энэ нөхцөлд 120 ширхэг сургалтын, 30 ширхэг туршилтын өгөгдөлд хуваагдана
for train_index, test_index in kf:
    X_train, y_train = data.records[train_index], data.targets[train_index]
    X_test, y_test = data.records[test_index], data.targets[test_index]
    
    clf_g = GaussianNB()
    clf_g.fit(X_train, y_train)
    pred = clf_g.predict(X_test)
    
    f1 = metrics.f1_score(y_test, pred, average='micro')
    f1_gauss.append(f1)
    
    clf_r = RandomForestClassifier(n_estimators=15)
    clf_r.fit(X_train, y_train)
    pred = clf_r.predict(X_test)
    
    f1 = metrics.f1_score(y_test, pred, average='micro')
    f1_rf.append(f1)
    
    clf_knn = KNeighborsClassifier(n_neighbors=5)
    clf_knn.fit(X_train, y_train)
    pred = clf_knn.predict(X_test)
    
    f1 = metrics.f1_score(y_test, pred, average='micro')
    f1_knn.append(f1)

In [9]:
#үр дүнг хэвлэх
print('Gaussian F1 Score: ', f1_gauss)
print('Random Forest F1 Score: ', f1_rf)
print('KNN F1 Score: ', f1_knn)

Gaussian F1 Score:  [0.8666666666666667, 0.9333333333333333, 0.9, 0.8666666666666667, 0.8666666666666667]
Random Forest F1 Score:  [0.9, 0.9666666666666667, 0.9, 0.9333333333333333, 0.9333333333333333]
KNN F1 Score:  [0.9666666666666667, 0.9333333333333333, 0.9666666666666667, 0.9666666666666667, 0.9666666666666667]


In [17]:
#ялгаатай хөршүүд дээр kNN алгоритмыг үнэлэх
index = 0
F1_SCORE = np.zeros(shape=(5, 4))
neighbors = [3, 5, 10, 20]
kf = cross_validation.KFold(len(data.records), n_folds=5, shuffle=True)
for train_index, test_index in kf:
    X_train, y_train = data.records[train_index], data.targets[train_index]
    X_test, y_test = data.records[test_index], data.targets[test_index]
        
    f1 = []    
        
    for n in neighbors:
        clf_knn = KNeighborsClassifier(n_neighbors=n)
        clf_knn.fit(X_train, y_train)
        pred = clf_knn.predict(X_test)
        
        f1.append(metrics.f1_score(y_test, pred, average='micro'))
        
    F1_SCORE[index] = f1
    index = index + 1